In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
#!pip install pylibjpeg pylibjpeg-libjpeg pydicom
#!pip install python-gdcm
#import pylibjpeg
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

os.mkdir('/kaggle/working/train/')



In [ ]:

train = pd.read_csv('/kaggle/input/siim-covid19-detection/train_image_level.csv')
train_study = pd.read_csv('/kaggle/input/siim-covid19-detection/train_study_level.csv')
train_study['StudyInstanceUID'] = train_study['id'].apply(lambda x: x.replace('_study', ''))
del train_study['id']
train = train.merge(train_study, on='StudyInstanceUID')
class_names = ['Negative for Pneumonia','Typical Appearance', 'Indeterminate Appearance','Atypical Appearance']
classMap={0:[1, 0, 0, 0],
         1:[0, 1, 0, 0],
         2:[0, 0, 1, 0],
         3:[0, 0, 0, 1]}

def findClass(value):
    value=value.tolist()
    for key,val in classMap.items():
        if value==val:
            return key
train["class"] =[findClass(li) for li in train[class_names].values]
train.to_csv('/kaggle/working/train.csv')

In [ ]:
filepath=[]
imgfile=[]
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection/train'):
    for filename in filenames:
        imgfile.append(os.path.join(dirname, filename))
        #print(dirname)
        img_path = os.path.join('/kaggle/working/train/',filename)
        #img_path = img_path.replace('.dcm', '.png')
        filepath.append(img_path)

#import shutil
#for i in range(0,1200):
        #os.system('med2image -i '+imgfile[i]+' -d '+filepath[i])
        #shutil.copy(imgfile[i], filepath[i])
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
from torchvision.transforms import ToTensor,Lambda
from torch.utils.data import DataLoader
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
model = models.resnet50(pretrained=False)

In [ ]:
for param in model.parameters():
    param.requires_grad = True
    
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 4),
                                 nn.Softmax(dim=1))
criterion =nn.MultiLabelSoftMarginLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
model.to(device)

In [ ]:
model.train()
for epoch in range(0,3):
    for i in range(0,1000):
        img_path=imgfile[i]
        dicom= pydicom.read_file(img_path)
        try:
            image = apply_voi_lut(dicom.pixel_array, dicom)
        except:
            print('exception')
            continue
        image = image - np.min(image)
        image = image / np.max(image)
        image = (image*255).astype(np.uint8)
        image=cv2.resize(image, dsize=(406, 406), interpolation=cv2.INTER_CUBIC)
        image=np.stack((image,)*3, axis=2)
        test_transforms = transforms.Compose([transforms.ToTensor(),])
        image_tensor = test_transforms(image)
        #image_tensor=torch.stack((image_tensor,)*64, axis=0)
        image_tensor = image_tensor.unsqueeze(0)
        inputData = image_tensor.to(device)
        logps = model.forward(inputData.type(torch.cuda.FloatTensor))
        img_id=img_path.split("/")
        img_id=img_id[len(img_id)-1].replace('.dcm','_image')
        labels=train[train['id']==img_id]['class']
        #print(labels)
        labels=torch.Tensor(list(labels)).to(device).type(torch.cuda.LongTensor)
        #print(logps)
        #print(labels)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        loss.item()
    
        
    

In [ ]:
torch.save(model, 'model.pth')


In [ ]:
model.eval()

In [ ]:
col=['id','PredictionString']
submission=pd.DataFrame(columns = col) 
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection/test'):
    for filename in filenames:
        img_path= os.path.join(dirname, filename)
        dicom= pydicom.read_file(img_path)
        try:
            image = apply_voi_lut(dicom.pixel_array, dicom)
        except:
            PredictionString= f'negative 0 0 0 1 1 typical 0 0 0 1 1 indeterminate 1 0 0 1 1 atypical 0 0 0 1 1'
            id_=img_path.split('/')[5]+"_study"
            submission=submission.append({"id":id_,"PredictionString":PredictionString},ignore_index=True)
            continue
        image = image - np.min(image)
        image = image / np.max(image)
        image = (image*255).astype(np.uint8)
        image=cv2.resize(image, dsize=(1984, 1984), interpolation=cv2.INTER_CUBIC)
        image=np.stack((image,)*3, axis=2)
        test_transforms = transforms.Compose([transforms.ToTensor(),])
        image_tensor = test_transforms(image)
        #image_tensor=torch.stack((image_tensor,)*64, axis=0)
        image_tensor = image_tensor.unsqueeze(0)
        inputData = image_tensor.to(device)

        output = model(inputData.type(torch.cuda.FloatTensor))
        index = output.data.cpu().numpy().argmax()
        predProb=output.data.cpu().numpy()[0]
        id_=img_path.split('/')[5]+"_study"
        #try:
            #PredictionString=submission[submission['id']==id_]['PredictionString'].iloc[0]+' '#.to_string()
        #except:
            #PredictionString=''
        PredictionString=f'negative {format(predProb[0],".8f")} 0 0 1 1 typical {format(predProb[1],".5f")} 0 0 1 1 indeterminate {format(predProb[2],".5f")} 0 0 1 1 atypical {format(predProb[3],".5f")} 0 0 1 1'           #PredictionString+class_names[index].split(' ')[0]+str(predProb[0])+' 0 0 1 1'
        #df2 = pd.DataFrame([[id_, PredictionString]], columns=col)
        submission=submission.append({"id":id_,"PredictionString":PredictionString},ignore_index=True)
        

Bounding box predictions using pretrained weights

In [ ]:
!cp ../input/yolov3/darknet/backup/darknet-yolov3_final.weights .
!cp ../input/yolov3/darknet/cfg/darknet-yolov3.cfg .
!cp ../input/yolov3/darknet/obj.names .

In [ ]:
import cv2
import numpy as np 
import argparse
import time
import os

net = cv2.dnn.readNet("/kaggle/working/darknet-yolov3_final.weights","/kaggle/working/darknet-yolov3.cfg")
classes = []
with open("/kaggle/working/obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layers_names = net.getLayerNames()
output_layers = [layers_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [ ]:
def get_box_dimensions(outputs, height, width):
	boxes = []
	confs = []
	class_ids = []
	for output in outputs:
		for detect in output:
			scores = detect[5:]
			
			class_id = np.argmax(scores)
			conf = scores[class_id]
			if conf > 0.3:
				center_x = int(detect[0] * width)
				center_y = int(detect[1] * height)
				w = int(detect[2] * width)
				h = int(detect[3] * height)
				x = int(center_x - w/2)
				y = int(center_y - h / 2)
				boxes.append([x, y, w, h])
				confs.append(float(conf))
				class_ids.append(class_id)
	return boxes, confs, class_ids

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection/test'):
    for filename in filenames:
        img_path= os.path.join(dirname, filename)
        dicom= pydicom.read_file(img_path)
        id_=img_path.split('/')[7].replace('.dcm','_image')
        try:
            image = apply_voi_lut(dicom.pixel_array, dicom)
        except:
            PredictionString='none 1 0 0 1 1'
            
            submission=submission.append({"id":id_,"PredictionString":PredictionString},ignore_index=True)
            continue
        image = image - np.min(image)
        image = image / np.max(image)
        image = (image*255).astype(np.uint8)
        img=cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
    
        height, width = img.shape
        blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(224, 224), mean=(0, 0, 0), swapRB=True, crop=False)
        net.setInput(blob)
        outputs = net.forward(output_layers)
        boxes, confs, class_ids=get_box_dimensions(outputs,224,224)
        PredictionString='none 1 0 0 1 1'
        submission=submission.append({"id":id_,"PredictionString":PredictionString},ignore_index=True)

In [ ]:
submission.to_csv('/kaggle/working/submission.csv',index=False)